# Introduction

This code is made to build a baseline in order to compare Capsnet performances vs Baseline along the different datasets. <br>
The baseline is a standard CNN with three convolutional layers of 256; 256; 128  channels. Each has 5x5 kernels and stride of 1. The last convolutional layers are followed by two fully connected layers of size 328; 192 . The last fully connected layer is connected with dropout to a 10  class softmax layer with cross entropy loss. <br><br>
The Model is trained without data augmentation methods on the regular MNIST dataset and then is tested with the different modified MNIST datasets. (The datasets are coming from LISA : http://www.iro.umontreal.ca/~lisa/twiki/bin/view.cgi/Public/MnistVariations). 

# Imports

In [42]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger
from keras.optimizers import RMSprop, Adam, SGD, Nadam


import numpy as np

# Data

In [34]:
#load train data set 

# the data, split between train and test sets
(x_train, y_train), _ = mnist.load_data()


if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_train /= 255
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print(x_train.shape[0], 'train samples')


x_train shape: (60000, 28, 28, 1)
y_train shape: (60000,)
60000 train samples


In [35]:
#load the test datasets


path_test = './data_transfo/'

test_normal = np.loadtxt(path_test + 'mnist_test.amat')
test_rotated = np.loadtxt(path_test + 'mnist_all_rotation_normalized_float_test.amat')
test_back_rand = np.loadtxt(path_test + 'mnist_background_random_test.amat')
test_back_img = np.loadtxt(path_test + 'mnist_background_images_test.amat')
test_rot_back_img = np.loadtxt(path_test + 'mnist_all_background_images_rotation_normalized_test.amat')

x_test = test_normal[:,:-1].reshape(-1,28,28,1)
y_test = test_normal[:,-1]

x_test_rotated = test_rotated[:,:-1].reshape(-1,28,28,1)
y_test_rotated = test_rotated[:,-1]

x_test_back_rand = test_back_rand[:,:-1].reshape(-1,28,28,1)
y_test_back_rand = test_back_rand[:,-1]

x_test_back_img = test_back_img[:,:-1].reshape(-1,28,28,1)
y_test_back_img = test_back_img[:,-1]

x_test_rot_back_img = test_rot_back_img[:,:-1].reshape(-1,28,28,1)
y_test_rot_back_img = test_rot_back_img[:,-1]


print('Test Shape', x_test.shape, y_test.shape)
print('Test Rotated Shape',x_test_rotated.shape, y_test_rotated.shape)
print('Test Back Rand Shape',x_test_back_rand.shape, y_test_back_rand.shape)
print('Test Back Img Shape',x_test_back_img.shape, y_test_back_img.shape)
print('Test Rot Back Img Shape',x_test_rot_back_img.shape, y_test_rot_back_img.shape)

Test Shape (12000, 28, 28, 1) (12000,)
Test Rotated Shape (12000, 28, 28, 1) (12000,)
Test Back Rand Shape (12000, 28, 28, 1) (12000,)
Test Back Img Shape (12000, 28, 28, 1) (12000,)
Test Rot Back Img Shape (12000, 28, 28, 1) (12000,)


In [36]:
#convert to categorical the labels 

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
y_test_rotated = keras.utils.to_categorical(y_test_rotated, num_classes)
y_test_back_rand = keras.utils.to_categorical(y_test_back_rand, num_classes)
y_test_back_img = keras.utils.to_categorical(y_test_back_img, num_classes)
y_test_rot_back_img = keras.utils.to_categorical(y_test_rot_back_img, num_classes)

# Parameters

In [43]:
#3 convolutional layers

conv1 = 256
conv2 = 256
conv3 = 128

#kernel size is 5 5 
kernel_size = 5
stride = 1

#2 fully connected layers
layer1 = 328
layer2 = 192

#drop proba for the dropout (in Keras we specify the droping probability/ in Tensorflow it's the keeping probability)
drop_prob = 0.2

#loss
loss = 'categorical_crossentropy'

learning = 2e-3


    
batch_size = 32
num_classes = 10
epochs = 12

opt = Nadam(lr=learning)

# input image dimensions
img_rows, img_cols = 28, 28

# Network

In [44]:
model = Sequential()

#convolutional layers
#layer1
model.add(Conv2D(conv1, kernel_size=(kernel_size, kernel_size),
                 activation='relu',
                 input_shape=input_shape))
#layer2
model.add(Conv2D(conv2, (kernel_size, kernel_size), activation='relu'))
#layer3
model.add(Conv2D(conv3, (kernel_size, kernel_size), activation='relu'))

#maxpooling 
#####don't know if we need
model.add(MaxPooling2D(pool_size=(2, 2)))

#drop out layer 
model.add(Dropout(drop_prob))

#fully connected layers
model.add(Flatten())
#layer1
model.add(Dense(layer1, activation='relu'))
#layer2
model.add(Dense(layer2, activation='relu'))

#dropout
model.add(Dropout(drop_prob))

#softmax layer
model.add(Dense(num_classes, activation='softmax'))



In [45]:

#optimization
#model.compile(loss=keras.losses.categorical_crossentropy,
#              optimizer=keras.optimizers.Adadelta(),
#              metrics=['accuracy'])

# #training
# model.fit(x_train, y_train,
#           batch_size=batch_size,
#           epochs=epochs,
#           verbose=1,
#           validation_data=(x_test, y_test))

# #testing
# score = model.evaluate(x_test, y_test, verbose=0)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])

The training is made with the regular MNIST.

In [46]:

#training

reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.9, patience=30, min_lr=0.000001, verbose=1)
checkpointer = ModelCheckpoint(filepath="baseline_mnist.hdf5", verbose=1, save_best_only=True)


model.compile(optimizer=opt, 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(x_train, y_train, 
          nb_epoch = epochs, 
          batch_size = batch_size, 
          verbose=1, 
          validation_data=(x_test, y_test),
          callbacks=[reduce_lr, checkpointer],
          shuffle=True)


/Users/Selim/anaconda/lib/python3.6/site-packages/keras/models.py:944: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 60000 samples, validate on 12000 samples
Epoch 1/12
  704/60000 [..............................] - ETA: 1:17:13 - loss: 2.1394 - acc: 0.2670

KeyboardInterrupt: 

Then, we test using the different versions of MNIST. 

In [49]:
# #testing

#model.load_weights("baseline_mnist.hdf5")

#test dataset
score_test = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score_test[0])
print('Test accuracy:', score_test[1])

#test dataset
score_test_rotated = model.evaluate(x_test_rotated, x_test_rotated, verbose=0)
print('Test Rotated loss:', score_test_rotated[0])
print('Test Rotated accuracy:', score_test_rotated[1])

#test dataset
score_test_back_rand = model.evaluate(x_test_back_rand, y_test_back_rand, verbose=0)
print('Test Back Rand loss:', score_test_back_rand[0])
print('Test Back Rand accuracy:', score_test_back_rand[1])

#test dataset
score_test_back_img = model.evaluate(x_test_back_img, y_test_back_img, verbose=0)
print('Test Back Img loss:', score_test_back_img[0])
print('Test Back Img accuracy:', score_test_back_img[1])

#test dataset
score_test_rot_back_img = model.evaluate(x_test_rot_back_img, y_test_rot_back_img, verbose=0)
print('Test Rot Back Img loss:', score_test_rot_back_img[0])
print('Test Rot Back Img accuracy:', score_test_rot_back_img[1])

  576/12000 [>.............................] - ETA: 4:22

KeyboardInterrupt: 